# Detection color area

Author : Johan Jublanc
    
Date : 06/11/2019

Description : 
- detect area with a particular color
- set bounding box around the area

In [ ]:
import json
import shutil
import random
import pathlib
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display as display

import mlflow
import mlflow.tensorflow
import mlflow.keras

import tensorflow as tf

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import MSE, MSLE
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from xebikart.images import transformer as T
import xebikart.dataset as dataset

from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

Eager Execution allows to evaluate operations immediately without building graphs
note : Only needed when not using TF 2.0

In [ ]:
tf.enable_eager_execution()

# Parameters

In [ ]:
tubes_root_folder = "file:/workspace/xebikart-ml-tubes"
test_size=0.2

# orange obstacle
#tubes_folders = ["tub.v9.orange"]
tubes_folders = ["tub.v9.pink"]

In [ ]:
#color_detected = [181, 126, 46]
color_detected = [185,91,117]
#color_detected = [67, 92, 62]

### Plot some examples

In [ ]:
raw_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders, tubes_extension=".tar.gz")
tubes_df = raw_tubes_df.rename(columns={"cam/image_array": "images_path", "user/angle": "angles", "user/throttle": "throttles"})
tubes_df.count()

fig, axs = plt.subplots(1, 3, figsize=(15,5), constrained_layout=True)
fig.suptitle("Angle / Throttle", fontsize=20)

for n, sample in tubes_df.sample(3).reset_index().iterrows():
    random_image_path = sample["images_path"]
    angle = sample["angles"]
    throttle = sample["throttles"]
    image = mpimg.imread(random_image_path) 
    axs[n].set_title(f"{angle} / {throttle}")
    axs[n].imshow(image)
    axs[n].get_xaxis().set_visible(False)
    axs[n].get_yaxis().set_visible(False)
print("size : {} images".format(len(raw_tubes_df)))

# 1/ Build a mask corresponding to the area

Here we detect pixels corresponding to the color chosen with a margin of error. To be more accurate, each pixel $pix_i$ is composed of 3 values between 0 and 255, one for each color channel (red, green, blue).

The formula we use is as follow : 
$$mask_i = \mathbb{1}_{pix_i(R) = color(R) +/- \epsilon}\times \mathbb{1}_{pix_i(G) = color(G) +/- \epsilon} \times \mathbb{1}_{pix_i(B) = color(B) +/- \epsilon}$$

Where : 
- $mask_i$ is the pixel i of the one-channel image output (one channel beacause we just need a black and white mask)
- $pix_i(R/G/B)$ value of the channel red/green/blue for the pixel $i$
- $color(R/G/B)$ value of the channel red/green/blue for the color detected.

That function keep the pixels that correspond to color that we want to detect for each RGB channel with amargin of error $\epsilon$. The other pixels are ignored. For the pixel kept we return 1 and for the other we return 0.

NB : for practical reason the following function inverse ones and zeros (we want to put the colored detected in black).

In [ ]:
def detect_color_area(tf_image_original, color_to_detect, epsilon):
    tf_boolean_channels = []
    
    for i in range(len(color_to_detect)):
        tf_ = tf.where(((tf_image_original[:,:,i] < (color_to_detect[i] + epsilon))&
                       (tf_image_original[:,:,i] > (color_to_detect[i] - epsilon))), 
                 tf.ones_like(tf_image_original[:,:,i]),
                 tf.zeros_like(tf_image_original[:,:,i]))
                 
        
        tf_boolean_channels.append(tf.expand_dims(tf_, axis=0, name=None))
        
    tf_sum = tf.math.reduce_sum(tf.concat(tf_boolean_channels, 0),0)
        
    return tf.where(tf_sum >= 3, tf.zeros_like(tf_sum), tf.ones_like(tf_sum))

## 1.1/ Plot somme examples

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15,5), constrained_layout=True)
for i in range(4):
    # function pre-defined are used to compute the prediction
    tf_image = T.read_image(tubes_df.sample()["images_path"].values[0])
    
    # eaxh image in shown with the prediction
    axs[1][i].imshow(detect_color_area(tf_image, color_to_detect = color_detected , epsilon=30), cmap="gray")
    axs[0][i].imshow(tf_image)

# 2/ Set a bounding box around the area

Here we sum detected pixel along each axis and keep the position for which at least $n$ pixels have been detected, where $n$ is a parameter. The for each axis we return the min and max positions where at least $n$ pixels have been detected.

In [ ]:
def bounding_shape_in_box(tf_binary_mask, nb_pixel_min):
    min_axis = []
    max_axis = []
    
    for i in range(2):
        threshold = tf_binary_mask.shape[i] - nb_pixel_min
        tf_sum_axisi = tf.math.reduce_sum(tf_binary_mask, axis=i)
        min_ = tf.math.reduce_min(tf.where(tf_sum_axisi <= int(threshold)))
        max_ = tf.math.reduce_max(tf.where(tf_sum_axisi <= int(threshold)))
        min_axis.append(min_/tf_binary_mask.shape[(1-i)**2])
        max_axis.append(max_/tf_binary_mask.shape[(1-i)**2])
    
    box = [min_axis[1], min_axis[0], max_axis[1], max_axis[0]]
    box = tf.expand_dims(tf.expand_dims(box,0),0)
    
    return tf.dtypes.cast(box,dtype = tf.float32)

In [ ]:
def return_image_and_box(image_path, color_detected, epsilon, nb_pixel_min):
    tf_image_original   = T.read_image(image_path)

    # Create a detection mask
    tf_color_area = detect_color_area(tf_image_original, color_detected, epsilon)
    
    # Create the boudning box
    box = bounding_shape_in_box(tf_color_area, nb_pixel_min)

    # Normilaze the original image
    tf_image_normalized = tf.image.convert_image_dtype(tf_image_original, dtype=tf.float32)
    tf_image_normalized = tf.expand_dims(tf_image_normalized, 0)
    
    return tf.image.draw_bounding_boxes(tf_image_normalized, box)[0,:,:,:]

## 2.1/ Plot some examples

In [ ]:
epsilon = 30
nb_pixel_min = 20

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(15,15), constrained_layout=True)

for i in range(12):
    random_image_path = tubes_df.sample()["images_path"].values[0]
    img = return_image_and_box(random_image_path, color_detected, epsilon, nb_pixel_min)
    axs[i//3][i%3].imshow(img)